In [1]:
import gym

In [2]:
env = gym.make('SpaceInvaders-v0')

In [3]:
# episodes = 10

# for episode in range(1, episodes):
#     state = env.reset()
#     done = False
#     score = 0
    
#     while not done:
#         env.render()
#         state, reward, done, info = env.step(env.action_space.sample())
#         score += reward
#     print('Episode: {}\nScore: {}'.format(episode,score))

# env.close()

In [3]:
#Import Neural Network Packages
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

from tensorflow.keras.optimizers import Adam

In [4]:
#Implementing a CNN with input of the screen pixels and output of actions by the agent
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides = (4,4), activation = 'relu', input_shape = (3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides = (2,2), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model
    

In [5]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [21]:
del model

In [22]:
model = build_model(height, width, channels, actions)

In [23]:
#Importing keras-rl2 reinforcement learning functions
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
# %config Completer.use_jedi = False

In [24]:
def build_agent(model, actions):
        policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr = 'eps', value_max=1., value_min=.1, value_test= .2, nb_steps= 10000)
        memory = SequentialMemory(limit=2000, window_length = 3)
        dqn = DQNAgent(model = model, memory = memory, policy=policy,
                      enable_dueling_network=True, dueling_type = 'avg',
                      nb_actions = actions, nb_steps_warmup =1000)
        return dqn

In [25]:
dqn = build_agent(model, actions)

In [26]:
dqn.compile(Adam(lr = 0.0001))

In [ ]:
#Run this block only if you want to retrain from scratch
dqn.fit(env, nb_steps = 40000, visualize = False, verbose = 1)

In [ ]:
#Run this block if you want to save weights after training
dqn.save_weights('models/dqn.h5f')

In [27]:
#Run this block if you want to load a pre-trained model
dqn.load_weights('models/dqn.h5f')

In [ ]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))